<a href="https://colab.research.google.com/github/M-H-Amini/Computer-Vision-in-Practice-Course/blob/main/CVinPractice_Lecture1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  In The Name Of ALLAH
#  Computer Vision in Practice
#  Mohammad Hossein Amini (mhamini@aut.ac.ir)
#  PythonChallenge.ir
#  Lecture 1 - Image Classification

#  Vanilla CNN

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist

##  Dataset

In [ ]:
(Xtrain, ytrain), (Xtest, ytest) = mnist.load_data()
Xtrain = np.expand_dims((Xtrain - 127.5) / 127.5, -1)
Xtest = np.expand_dims((Xtest - 127.5) / 127.5, -1)
print(Xtrain.shape, ytrain.shape)
print(Xtest.shape, ytest.shape)

In [ ]:
def show(X, y):
  plt.figure()
  plt.imshow(X[:, :, 0], cmap='gray')
  plt.title(f'Target = {y}')
  plt.show()

show(Xtrain[10], ytrain[10])

##  Model

In [ ]:
model = Sequential()
model.add(Conv2D(32, 3, activation='relu', padding='same', input_shape=(28, 28, 1)))
model.add(MaxPool2D())
model.add(Conv2D(64, 3, activation='relu', padding='same'))
model.add(MaxPool2D())
model.add(Conv2D(128, 3, activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(32, activation='tanh'))
model.add(Dense(10, activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(Xtrain, ytrain, batch_size=32, epochs=10, validation_data=(Xtest, ytest))

In [ ]:
img = Xtest[10:11]
plt.imshow(img[0, :, :, 0])
output = model(img)
print(np.argmax(output))

#  Covid-19 Detection

In [ ]:
from google.colab import drive
from PIL import Image
import os
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.models import Model
drive.mount('/content/gdrive')

##  Dataset

In [ ]:
!cp /content/gdrive/My\ Drive/covid/kaggle.json kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json
#!kaggle datasets list
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database

In [ ]:
!unzip -q covid19-radiography-database.zip -d covid

In [ ]:
covid_path = os.path.join('covid', 'COVID-19_Radiography_Dataset', 'COVID')
normal_path = os.path.join('covid', 'COVID-19_Radiography_Dataset', 'Normal')
covid_files = [os.path.join(covid_path, file) for file in os.listdir(covid_path)]
normal_files = [os.path.join(normal_path, file) for file in os.listdir(normal_path)]

In [ ]:
# m = min(len(covid_files), len(normal_files))
m = 1500

In [ ]:
covid_images = np.array([np.array(Image.open(image).convert('RGB').resize((224, 224))) for image in covid_files[:m]])
normal_images = np.array([np.array(Image.open(image).convert('RGB').resize((224, 224))) for image in normal_files[:m]])

print(covid_images.shape)
print(normal_images.shape)

In [ ]:
##  How many to take for training part...
# N = 3200
N = 1200

In [ ]:
covid_train, covid_test = covid_images[:N], covid_images[N:]
normal_train, normal_test = normal_images[:N], normal_images[N:]
covid_train_targets, covid_test_targets = np.ones((len(covid_train))), np.ones((len(covid_test)))
normal_train_targets, normal_test_targets = np.zeros((len(normal_train))), np.zeros((len(normal_test)))
print(covid_train.shape, covid_test.shape)
print(normal_train.shape, normal_test.shape)

In [ ]:
Xtrain = np.concatenate((covid_train, normal_train))
ytrain = np.concatenate((covid_train_targets, normal_train_targets))
Xtest = np.concatenate((covid_test, normal_test))
ytest = np.concatenate((covid_test_targets, normal_test_targets))
print(Xtrain.shape, ytrain.shape)
print(Xtest.shape, ytest.shape)

In [ ]:
Xtrain = preprocess_input(Xtrain)
Xtest = preprocess_input(Xtest)

In [ ]:
plt.figure()
plt.imshow(Xtrain[10])
plt.title(f'Target: {ytrain[10]}')

##  Model

In [ ]:
base = VGG19(include_top=False, weights='imagenet', input_shape=(224,224,3))
x = base.output
x = Flatten()(x)
x = Dense(16, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

for layer in base.layers:
  layer.trainable = False

model = Model(inputs=base.input, outputs=x)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='bce', metrics=['acc'])
model.fit(Xtrain, ytrain, batch_size=32, epochs=10, validation_data=(Xtest, ytest))